In [1]:
print("ok")

ok


In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\suraj\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY="14be14a9-7407-486e-8c0b-e4d9d0b3e320"
PINECONE_API_ENV="us-east-1"

In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
#pip install pypdf

In [5]:
extracted_data=load_pdf("C:\medical chatbot\End-to-End-Medical-chatbot-using-Lamma2\data")

In [6]:
#extracted_data

In [7]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks=text_split(extracted_data)
print(len(text_chunks))

7020


In [9]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings=download_hugging_face_embeddings()

In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [12]:
query_result=embeddings.embed_query("hello world")
print("length",len(query_result))

length 384


In [13]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name="medical-bot"

#Loading the index


In [14]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
pc.list_indexes()


{'indexes': [{'dimension': 1536,
              'host': 'new-index-rrj4zyv.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'new-index',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 8,
              'host': 'docs-overview-index-rrj4zyv.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'docs-overview-index',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 384,
              'host': 'medical-chatbot-rrj4zyv.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'medical-chatbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [15]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "new-index"
if index_name not in pc.list_indexes().names():
  # Do something, such as create the index
  pc.create_index(
    name=index_name,
    dimension=1536,
    metric='cosine',
    spec=ServerlessSpec(
      cloud="aws",
      region="us-east-1"
    )
  )


In [16]:
#pip install --upgrade pinecone-client

In [17]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

pc.list_collections()
# Response:
# {'dimension': 1536,
#  'host': 'serverless-index-4zo0ijk.svc.us-weset2-aws.pinecone.io',
#  'metric': 'cosine',
#  'name': 'serverless-index',
#  'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
#  'status': {'ready': True, 'state': 'Ready'}}


{'collections': []}

In [18]:
#pip install -U langchain-pinecone

In [19]:
index_name="medical-chatbot"

In [20]:
index = pc.Index(index_name)

In [21]:
index.query(
    namespace="ns1",
    vector=[0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
    top_k=3,
    include_values=True
)

index.query(
    namespace="ns2",
    vector=[0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7],
    top_k=3,
    include_values=True
)


{'matches': [], 'namespace': 'ns2', 'usage': {'read_units': 0}}

In [22]:
from pinecone import Pinecone

# Initialize Pinecone with your API key
pinecone = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-chatbot"

# Retrieve information about the specified index
index_info = pinecone.describe_index(index_name)

# Print the retrieved index information
print(index_info)


{'dimension': 384,
 'host': 'medical-chatbot-rrj4zyv.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'medical-chatbot',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}


In [24]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

In [30]:
# Specify the name of your existing index
index_name = "medical-chatbot"
index = pc.Index(index_name)


In [37]:
spec = ServerlessSpec(cloud=cloud, region=region)

NameError: name 'cloud' is not defined

In [38]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get("14be14a9-7407-486e-8c0b-e4d9d0b3e320") 

# configure client
pc = Pinecone(api_key=api_key)

PineconeConfigurationError: You haven't specified an Api-Key.